# Making Timing Files for Event-Related fMRI Study

Author: Isaac Cormier
Date:2024-06-24

## CreateTiming Function

In [132]:
##TODO:Remove the inclusion of the TR and approx_time_sec collumns for the timing text files once confirmed that they are correct

import os
import pandas as pd
import numpy as np

def createTiming(directory, condition=['Picture', 'Word', 'Face'], ids = range(1,31)):
    #Check that condition is valid
    condition_list = ['Picture', 'Word', 'Face']
    condition = [c.capitalize() for c in condition]
    if not any(cond in condition_list for cond in condition):
        raise ValueError("invalid condition")
        
    #Load participant data and remove rows with blank TRs
    for i in ids:
        zero_id = f"{i:03}"
        csv = os.path.join(os.getcwd(), directory, f"fMRI_MEG_{zero_id}_memoryTest_responses_withCodes.csv")
        df = pd.read_csv(csv, usecols=[0,1,2,3,4,5,6,7])
        df.rename(columns={
            'Stim Type': 'stim_type',
            'Stim': 'stim',
            'PostScan Trial': 'postscan_trial',
            'Response': 'response',
            'Correct Answer': 'correct_answer',
            'Code': 'code',
            'TR': 'TR',
            'Approx time (s)': 'approx_time_sec'
        }, inplace=True)
        df.dropna(subset=['TR'], inplace=True)
        df = df.sort_values('TR')
        if len(df) != 210:
            raise ValueError(f"There are {len(df)} number of rows, when there should be 210.")
            
        #Add rows corresponding to TRs not included for the stimuli
        rows_with_TRs = []
        for index, row in df.iterrows():
            rows_with_TRs.append(row) 
            new_row = row.copy()
            new_row.iloc[6] += 1
            new_row.iloc[7] += 2
            new_row.iloc[1:4] = np.nan
            rows_with_TRs.append(new_row)
        df_TRs = pd.DataFrame(rows_with_TRs)
        
        #Add rows corresponding to the fixation crosshairs at the start and between blocks
        rows_with_crosshair = []
        count = 0
        for index, row in df_TRs.iloc[::-1].iterrows():
            rows_with_crosshair.append(row)

            count += 1
            if count % 60 == 0:
                for x in range(10):
                    new_row = pd.Series({'stim_type': f"Crosshair",
                                         'stim': np.nan,
                                         'postscan_trial': np.nan,
                                         'response': np.nan,
                                         'correct_answer': np.nan,
                                         'code': 0,
                                         'TR': np.nan,
                                         'approx_time_sec': np.nan})
                    rows_with_crosshair.append(new_row)
        rows_with_crosshair.reverse()
        df_TRs_crosshair = pd.DataFrame(rows_with_crosshair[5:])

        #Create timing text files
        for p in condition:
            #Correctly remembered txt file (1=correctly remembered; 0=anything else)
            df_temp1 = df_TRs_crosshair.copy()
            df_temp1.loc[(df_temp1['stim_type'] != p) | (df_temp1['code'] != 1), 'code'] = 0
            correct = df_temp1[['code', 'TR', 'approx_time_sec']]
            output_txt1= os.path.join(os.getcwd(),directory, f"sub-{zero_id}_{p}_correctly_remembered.txt")
            correct.to_csv(output_txt1, sep='\t', index=False, header=False)
            
            #Incorrectly remembered txt file (1=incorrectly remembered; 0=anything else)
            df_temp2 = df_TRs_crosshair.copy()
            df_temp2.loc[(df_temp2['stim_type'] != p) | (df_temp2['code'] != 4), 'code'] = 0
            df_temp2.loc[(df_temp2['code']) == 4, 'code'] = 1
            incorrect = df_temp2[['code', 'TR', 'approx_time_sec']]
            output_txt2= os.path.join(os.getcwd(),directory, f"sub-{zero_id}_{p}_incorrectly_remembered.txt")
            incorrect.to_csv(output_txt2, sep='\t', index=False, header=False)    

## Example of Running CreateTiming Function

In [137]:
createTiming(directory="/Volumes/CIHR_STUDY/fmri_timing_files/original_data/",ids=[3])